In [2]:
#pip install pyproj
#pip install netdcf4
import os
import netCDF4 as nc
import numpy as np
import matplotlib.pyplot as plt
import pdb
import scipy.spatial.qhull as qhull
from scipy.interpolate import griddata
import pyproj
from u_interpolate_small import regrid_irregular_quick
from datetime import date
from ndays import numOfDays
import glob
#from mpl_toolkits.basemap import Basemap

In [3]:
# Define domain and time period
start_lat = -37
end_lat = -10
start_lon = -18
end_lon = 30
start_year = '2006'
start_month = '11'
end_year = '2006'
end_month = '12'
start_day = '01'
end_day = '31'

In [4]:
# find indices using one file
core_filename = '/prj/Africa_cloud/ch9_wavelet/2010/09/201009081045.nc'
core_data = nc.Dataset(core_filename,'r')
# read in cores lat/lon
lat = np.array(core_data.variables['lat'][:])
lon = np.array(core_data.variables['lon'][:])
#lon_filter = lon >-181 # acceptable values
#lon_fl = lon[lon_filter] #filtered long
lat_ind = np.where((lat>=start_lat) & (lat<=end_lat))[0]
lon_ind = np.where((lon>= start_lon) & (lon<=end_lon))[0]
lat = lat[lat_ind[0]:lat_ind[-1]]
lon = lon[lon_ind[0]:lon_ind[-1]]
#lon = lon_fl[(lon_fl>= start_lon) & (lon_fl<=end_lon)]
X, Y = np.meshgrid(lon,lat)

# find ndays
date1 = date(int(start_year), int(start_month), int(start_day))
date2 = date(int(end_year), int(end_month), int(end_day))
t = numOfDays(date1, date2)+1
#time = 
cores = np.zeros((len(lat),len(lon),t*24),dtype=float) #using every fourth value for hourly comparison- 24hr
#time = np.zeros((len(lat)-1,len(lon)-1,t*96))

In [5]:
# read in cores data
list_of_files = []

if int(start_year)==int(end_year): 
    for m in range(int(start_month),int(end_month)+1,1): # 1 to include the end month
        if m<10:
            month = str(m).zfill(1)
        else:
            month = str(m)               
        dir_name = '/prj/Africa_cloud/ch9_wavelet/'+start_year+'/'+month+'/'       
        all_file_names=sorted(glob.glob(dir_name+start_year+month+"*30.nc"));
            #print(all_files)
        list_of_files.append(all_file_names) # all days in month
             
elif int(start_year)<int(end_year):
    y=1
    for i in range(int(start_year),int(end_year)+1,1):
        #months_array = np.array(range(int(start_month),12+1,1))
        #months_array= np.append(months_array,range(1,int(end_month)+1,1))
        if y==1:
            for m in range(int(start_month),12+1,1):
                if m<10:
                    month = str(m).zfill(2)
                else:
                    month = str(m)  
                dir_name = '/prj/Africa_cloud/ch9_wavelet/'+str(i)+'/'+month+'/'       
                all_file_names=sorted(glob.glob(dir_name+str(i)+month+"*30.nc"));
                #print(all_file_names)
                list_of_files.append(all_file_names) # all days in month  
                y=y+1
                month=[]
            
        else:
            for m in range(1,int(end_month)+1,1):
                if m<10:
                    month = str(m).zfill(2)
                else:
                    month = str(m)    
                dir_name = '/prj/Africa_cloud/ch9_wavelet/'+str(i)+'/'+month+'/'       
                all_file_names2=sorted(glob.glob(dir_name+str(i)+month+"*30.nc"));
                list_of_files.append(all_file_names2) # all days in month 
                
                
#for a in len(list_of_files):
 #   list_of_files = list_of_files+list_of_files[a]
list_of_files = list_of_files[0]+list_of_files[1]
#print((list_of_files))

In [ ]:
#list_of_files = sorted(os.listdir())
counter = 0
for l in range(0,len(list_of_files),1):#2, min(t*96,len(list_of_files)), 4):#range(len(list_of_files)): # 96 files per day
    core_filename = list_of_files[l]
    print(core_filename)
#    core_filename = '/prj/Africa_cloud/ch9_wavelet/'+start_year+'/'+start_month+'/'+list_of_files[l]
    core_data = nc.Dataset(core_filename,'r')
    #print(core_data)
    cores_temp = np.array(core_data.variables['cores'])[0,lat_ind[0]:lat_ind[-1],lon_ind[0]:lon_ind[-1]]
    cores[:,:,counter] = cores_temp   
    counter = counter+1
    core_data.close()           

/prj/Africa_cloud/ch9_wavelet/2006/11/200611010030.nc
/prj/Africa_cloud/ch9_wavelet/2006/11/200611010130.nc
/prj/Africa_cloud/ch9_wavelet/2006/11/200611010230.nc
/prj/Africa_cloud/ch9_wavelet/2006/11/200611010330.nc
/prj/Africa_cloud/ch9_wavelet/2006/11/200611010430.nc
/prj/Africa_cloud/ch9_wavelet/2006/11/200611010530.nc
/prj/Africa_cloud/ch9_wavelet/2006/11/200611010630.nc
/prj/Africa_cloud/ch9_wavelet/2006/11/200611010730.nc
/prj/Africa_cloud/ch9_wavelet/2006/11/200611010830.nc
/prj/Africa_cloud/ch9_wavelet/2006/11/200611010930.nc
/prj/Africa_cloud/ch9_wavelet/2006/11/200611011030.nc
/prj/Africa_cloud/ch9_wavelet/2006/11/200611011130.nc
/prj/Africa_cloud/ch9_wavelet/2006/11/200611011230.nc
/prj/Africa_cloud/ch9_wavelet/2006/11/200611011330.nc
/prj/Africa_cloud/ch9_wavelet/2006/11/200611011430.nc
/prj/Africa_cloud/ch9_wavelet/2006/11/200611011530.nc
/prj/Africa_cloud/ch9_wavelet/2006/11/200611011630.nc
/prj/Africa_cloud/ch9_wavelet/2006/11/200611011730.nc
/prj/Africa_cloud/ch9_wavele

/prj/Africa_cloud/ch9_wavelet/2006/11/200611070930.nc
/prj/Africa_cloud/ch9_wavelet/2006/11/200611071030.nc
/prj/Africa_cloud/ch9_wavelet/2006/11/200611071130.nc
/prj/Africa_cloud/ch9_wavelet/2006/11/200611071230.nc
/prj/Africa_cloud/ch9_wavelet/2006/11/200611071330.nc
/prj/Africa_cloud/ch9_wavelet/2006/11/200611071430.nc
/prj/Africa_cloud/ch9_wavelet/2006/11/200611071530.nc
/prj/Africa_cloud/ch9_wavelet/2006/11/200611071630.nc
/prj/Africa_cloud/ch9_wavelet/2006/11/200611071730.nc
/prj/Africa_cloud/ch9_wavelet/2006/11/200611071830.nc
/prj/Africa_cloud/ch9_wavelet/2006/11/200611071930.nc
/prj/Africa_cloud/ch9_wavelet/2006/11/200611072030.nc
/prj/Africa_cloud/ch9_wavelet/2006/11/200611072130.nc
/prj/Africa_cloud/ch9_wavelet/2006/11/200611072230.nc
/prj/Africa_cloud/ch9_wavelet/2006/11/200611072330.nc
/prj/Africa_cloud/ch9_wavelet/2006/11/200611080030.nc


In [ ]:
# find precip indices using one file
precip_filename = '/prj/global_water/MCS_Feng/global_v2/2d_fields/20100101.0000_20110101.0000/mcstrack_20100901_0630.nc'
precip_data = nc.Dataset(precip_filename,'r')
lat_p = np.array(precip_data.variables['latitude'][:])
lon_p = np.array(precip_data.variables['longitude'][:])
lat_ind_p = np.where((lat_p[:,1]>=start_lat) & (lat_p[:,1]<=end_lat))[0]
lon_ind_p = np.where((lon_p[1,:]>= start_lon) & (lon_p[1,:]<=end_lon))[0]
lat_p = lat_p[lat_ind_p[0]:lat_ind_p[-1],lon_ind_p[0]:lon_ind_p[-1]]
lon_p = lon_p[lat_ind_p[0]:lat_ind_p[-1],lon_ind_p[0]:lon_ind_p[-1]]
#X_p, Y_p = np.meshgrid(lon_p,lat_p)

In [ ]:
# read in precip data 
list_of_files_precip=[]

if int(start_year)==int(end_year): 
    dir_name_precip =  '/prj/global_water/MCS_Feng/global_v2/2d_fields/'+start_year+'0101.0000_'+str(int(start_year)+1)+'0101.0000'
    os.chdir(dir_name_precip)
    for m in range(int(start_month),int(end_month)+1,1):
            if m<10:
                month = str(m).zfill(1)
            else:
                month = str(m)   
            all_files=sorted(glob.glob("mcstrack_"+start_year+month+"*.nc"));
            #print(all_files)
            list_of_files_precip.append(all_files) # all days in month
             
elif int(start_year)<int(end_year):
    y=1
    for i in range(int(start_year),int(end_year)+1,1):
        dir_name_precip =  '/prj/global_water/MCS_Feng/global_v2/2d_fields/'+str(i)+'0101.0000_'+str(i+1)+'0101.0000/'
        #os.chdir(dir_name_precip)
        if y==1:
            for m in range(int(start_month),12+1,1):
                if m<10:
                    month = str(m).zfill(2)
                else:
                    month = str(m)  
                list_of_files_precip.append(sorted(glob.glob(dir_name_precip+'mcstrack_'+str(i)+month+'*.nc'))) # day 1
                y=y+1            
        else:
            for m in range(1,int(end_month)+1,1):
                if m<10:
                    month = str(m).zfill(2)
                else:
                    month = str(m) 
                dir_name_precip =  '/prj/global_water/MCS_Feng/global_v2/2d_fields/'+str(i)+'0101.0000_'+str(i+1)+'0101.0000/'
                list_of_files_precip.append(sorted(glob.glob(dir_name_precip+'mcstrack_'+str(i)+month+'*.nc'))) # day 1
                    
    
    #print(dir_name_precip)
list_of_files_precip = list_of_files_precip[0]+list_of_files_precip[1]

In [ ]:
# prespecify
precip = np.zeros((len(lat_p[:,1]),len(lon_p[1,:]),t*24),dtype=float)
regidded_precip = np.zeros((len(lat),len(lon),t*24),dtype=float)
     
for l in range(0,len(list_of_files_precip),1):
    precip_filename = list_of_files_precip[l]
    print(precip_filename)
    precip_data = nc.Dataset(precip_filename,'r')
    precip_temp = np.array(precip_data.variables['precipitation'])[0,lat_ind_p[0]:lat_ind_p[-1],lon_ind_p[0]:lon_ind_p[-1]]
    precip[:,:,l] = precip_temp                                                                                                                                                            
    A = regrid_irregular_quick(lon_p[1,:],lat_p[:,1],lon,lat,precip_temp)
    regidded_precip[:,:,l] = A   

In [ ]:
# visualize core data at inherent grid/resol
plt.figure(figsize=(10,6))
plt.contourf(X,Y,cores[:,:,10],cmap='jet')
plt.colorbar(label='power')
plt.xlabel('Longitude')
plt.ylabel('Latitude')
plt.show()

# visualize core data at inherent grid/resol
plt.figure(figsize=(10,6))
plt.contourf(lon_p,lat_p,precip[:,:,10],cmap='jet')#
plt.colorbar(label='precip [mm/hr]')
plt.xlabel('Longitude')
plt.ylabel('Latitude')
plt.show()

# visualize core data at inherent grid/resol
plt.figure(figsize=(10,6))
plt.contourf(X,Y,regidded_precip[:,:,10],cmap='jet')#
plt.colorbar(label='precip [mm/hr]')
plt.xlabel('Longitude')
plt.ylabel('Latitude')
plt.show()

In [ ]:
# find rainfall pixels collocated with cores
cores_1D = cores.flatten()
precip_1D = regidded_precip.flatten() # cores and precip have same size and indexing - obs in connis notebook
cores_ind_gt_0 = np.where(cores_1D>0)[0]
precip_collocated_w_cores = precip_1D[cores_ind_gt_0] #dat in connis notebook
print(precip_collocated_w_cores.shape)
print(max(precip_collocated_w_cores))

In [ ]:
# calculate frequency of rainfall above a threshold 
# dat = 
# threshold = 30mm/hr

def calc_treshReached(dat, thresh):
    return np.sum(np.array(dat)>thresh)/np.sum(np.isfinite(dat))

In [ ]:
# calculate frequency of rainfall above a threshold relative to the same frequency for the full rainfall field (native)

def calc_treshReached_vsObs(dat, obs, thresh):
    return np.sum(np.array(dat)>thresh)/np.sum(np.isfinite(dat)) / np.sum(np.array(obs)>thresh)/np.sum(np.isfinite(obs))

In [ ]:
# calculate fraction of caught rainfall pixels above a threshold
# calc_treshMissed in connis script

def calc_treshcaught(dat, obs, thresh):
    return np.sum(np.array(dat)>thresh)/np.sum(np.array(obs)>thresh)

In [ ]:
# calculate frequency of rainfall above a threshold for regions with wavelet power above a given percentile

def percentile(dat, wav, thresh, perc):
    pval = np.percentile(wav, perc)
    dats = np.array(dat)[np.array(wav)>=pval]
    return np.sum(dats>thresh)/np.sum(np.isfinite(dats))

In [ ]:
# fraction of pixels where threshold exceeded
# dat = precip_collocated_w_cores
# obs = precip_1D
thresh = 20
# treached = calc_treshReached(precip_collocated_w_cores,thresh)
# print(treached)

In [ ]:
# fraction of caught pixels
# tcaught = calc_treshcaught(precip_collocated_w_cores,precip_1D, thresh)
# print(tcaught)

In [ ]:
# fraction of threshold reached in dat vs threshold reached in obs
tfrac = calc_treshReached_vsObs(precip_collocated_w_cores,precip_1D, thresh)
print(tfrac)

In [ ]:
# fraction of rainfall pixels reached above intensity threshold to all rainfall pixels (collocated with cores)
f = plt.figure(figsize=(6,4), dpi=200)
ax = f.add_subplot(111)

treached = []
for intens in range(5,50,5):   
    cc = calc_treshReached(precip_collocated_w_cores, intens)
    treached.append(cc)

ax.plot(range(5,50,5), treached,color='green')
ax.set_title("Intensity probability (%) (x[treshhold]/x)")
plt.xlabel('intensity of rainfall [mm/hr]')

In [ ]:
# plot frequency of caught pixels with rainfall above a threshold for regions with wavelet power above a given 
# percentile
f = plt.figure(figsize=(6,4), dpi=200)
ax = f.add_subplot(111)

treached = []
for intens in range(5,50,5):   
    cc = percentile(precip_collocated_w_cores, cores_1D[cores_ind_gt_0], intens, 75)
    treached.append(cc)
    #plotdic.append((treached,dat[1]))
#for pp in plotdic:    
#ax.plot(range(5,50,5), pp[0], label=pp[1])

ax.plot(range(5,50,5), treached,color='green')
ax.set_title("Intensity probability for 75th centile power (%) (x[treshhold]/x)")
plt.xlabel('intensity of rainfall [mm/hr]')
#ax.ylabel('fraction of rainfall pixels caught above intensity threshold that have >= 75th percentile wavelet power')
#plt.legend()
#f.savefig("/home/ck/DIR/cornkle/data/nflics/core_tests/"+'rainP_75p_wpower.jpg')

In [ ]:
# fraction of rainfall pixels caught above intensity threshold to all caught rainfall pixels
f = plt.figure(figsize=(6,4), dpi=200)
ax = f.add_subplot(111)

tcaught = []
for intens in range(5,50,5):   
    cc = calc_treshcaught(precip_collocated_w_cores, precip_1D, intens)
    tcaught.append(cc)

ax.plot(range(5,50,5), tcaught,color='red')
ax.set_title("Retained pixels per intensity (%)")
plt.xlabel('intensity of rainfall [mm/hr]')
#f.savefig("/home/ck/DIR/cornkle/data/nflics/core_tests/"+'_missedCases_southernAfricaDomain.jpg')   

In [ ]:
# read cores daily data
#core_filename = '/prj/Africa_cloud/ch9_wavelet/2010/06/201006081045.nc'
#core_filename = '/prj/Africa_cloud/ch9_wavelet/2010/',month,'/201006081045.nc'
#core_data = nc.Dataset(core_filename,'r')
#print(core_data)
#cores_temp = np.array(core_data.variables['cores'])
#cores = np.array(core_data.variables['cores'])
#time = core_data.variables['time'][:]

#cores = cores.append(cores_temp)
# make a lat x lon x time dataset for monthly cores
#global_attributes = core_data.__dict__


# read precip data
#precip_filename = '/prj/global_water/MCS_Feng/global_v2/2d_fields/'
#precip_data = nc.Dataset(precip_filename,'r')

# visualize core data at inherent grid/resol
#plt.figure(figsize=(10,6))
#plt.contourf(X,Y,np.max(cores,axis=2),cmap='jet')
#plt.colorbar(label='power')
#plt.title(cores.time)
#plt.xlabel('Longitude')
#plt.ylabel('Latitude')
#plt.show()

# visualize regridded data
# plt.figure(figsize=(10,6))
#plt.contourf(X_new,Y_new,A,cmap='jet')
#plt.colorbar(label='power')
#plt.title(cores.time)
#plt.xlabel('Longitude')
#plt.ylabel('Latitude')
#plt.show()

In [ ]:
# replace with precip data grid - bring coarse precip data (0.1) to fine core grid (0.04)
#new_lat = np.linspace(min(lat),max(lat),2000)#(lat[2]-lat[1])*2)
#new_lon = np.linspace(min(lon),max(lon),1900)#(lat[2]-lat[1])*2)
#X_new, Y_new = np.meshgrid(new_lon,new_lat)
#A = regrid_irregular_quick(lat_p,lon_p,lon,lat,cores_temp)
#regidded_precip[:,:,l] = A    